In [1]:
import sympy as sy
from sympy import sqrt, ln, exp
from sympy.printing.numpy import NumPyPrinter
def norm(x):
    dim = x.shape[0]
    z = 0
    for i in range(dim):
        z += x[i,0]**2
    return sy.sqrt(z)

def s(v):
    if v > 0:
        return 0
    else:
        return 1

In [11]:
# ファブリック　目標アトラクタ
m_u, m_l, alpha_m, k, alpha_psi = sy.symbols('m_u, m_l, alpha_m, k, alpha_psi')
M_s, xi_s, grad_psi_1_s, pi_s = [], [], [], []
for dim in [1, 2, 3]:
    print(dim)
    if dim == 1:
        x, x_dot = sy.symbols("x, x_dot")
        x_norm = sqrt(x**2)
        G = (m_u - m_l) * sy.exp(-(alpha_m * x_norm)**2) + m_l
    else:
        x = sy.MatrixSymbol('x', dim, 1)
        x_dot = sy.MatrixSymbol('x_dot', dim, 1)
        x_norm = norm(x)
        G = (m_u - m_l) * sy.exp(-(alpha_m * x_norm)**2) * sy.eye(dim) + m_l * sy.eye(dim)
    psi_1 = k * (x_norm + 1/alpha_psi * sy.ln(1 + sy.exp(-2 * alpha_psi * x_norm)))

    if dim == 1:
        L = x_dot**2 * G
    else:
        
        L = (x_dot.T * G * x_dot)[0,0]
    M = G
    M_s.append(M)

    if dim == 1:
        xi = sy.diff(sy.diff(L, x_dot), x) * x_dot - sy.diff(L, x)
        grad_psi_1 = sy.diff(grad_psi_1, x)
    elif dim == 2:
        xi = sy.Matrix([[sy.diff(L, x_dot[0,0]), sy.diff(L, x_dot[1,0])]]).T.jacobian(x) * x_dot \
            - sy.Matrix([[sy.diff(L, x[0,0])], [sy.diff(L, x[1,0])],])
        grad_psi_1 = sy.Matrix([[sy.diff(psi_1, x[0,0]), sy.diff(psi_1, x[1,0])]]).T
    else:
        xi = sy.Matrix([[sy.diff(L, x_dot[0,0]), sy.diff(L, x_dot[1,0]), sy.diff(L, x_dot[2,0])]]).T.jacobian(x) * x_dot \
            - sy.Matrix([[sy.diff(L, x[0,0])], [sy.diff(L, x[1,0])], [sy.diff(L, x[2,0])]])
        grad_psi_1 = sy.Matrix([[sy.diff(psi_1, x[0,0]), sy.diff(psi_1, x[1,0]), sy.diff(psi_1, x[2,0])]]).T
    pi = -M * grad_psi_1
    
    xi_s.append(xi)
    grad_psi_1_s.append(grad_psi_1)
    pi_s.append(pi)




with open("fabric_goal_attractor.py", "w") as f:
    for i in range(3):
        f.write("import numpy as np\nfrom numba import njit\n@njit(cache=True)\ndef xi_{0}(x, x_dot, m_u, m_l, alpha_m, k, alpha_psi):\n    return ".format(i+1))
        f.write(NumPyPrinter().doprint(xi_s[i]))
        f.write("\n@njit(cache=True)\ndef grad_psi_1_{0}(x, x_dot, m_u, m_l, alpha_m, k, alpha_psi):\n    return ".format(i+1))
        f.write(NumPyPrinter().doprint(grad_psi_1_s[i]))
        f.write("\n@njit(cache=True)\ndef pi_{0}(x, x_dot, m_u, m_l, alpha_m, k, alpha_psi):\n    return ".format(i+1))
        f.write(NumPyPrinter().doprint(pi_s[i]))
        f.write("\n@njit(cache=True)\ndef M_{0}(x, x_dot, m_u, m_l, alpha_m, k, alpha_psi):\n    return ".format(i+1))
        f.write(NumPyPrinter().doprint(M_s[i]))


1
2
3


In [ ]:
# 移動空間制限
a, wu, wl, sigma, r = sy.symbols('a, wu, wl, sigma, r')
x, x_dot = sy.symbols('x, x_dot')